In [1]:
#DATASET 
!wget http://cb.lk/covid_19

--2021-04-23 14:26:35--  http://cb.lk/covid_19
Resolving cb.lk (cb.lk)... 159.89.170.74
Connecting to cb.lk (cb.lk)|159.89.170.74|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cb.lk/covid_19 [following]
--2021-04-23 14:26:36--  https://cb.lk/covid_19
Connecting to cb.lk (cb.lk)|159.89.170.74|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/s/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip?dl=0 [following]
--2021-04-23 14:26:37--  https://www.dropbox.com/s/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/7rjw6oet4za01op/CovidDataset-20200427T133042Z-001.zip [following]
--2021-04-23 14:26:37--  https://www.dropbox.com/s/raw/7

In [2]:
!unzip covid_19

Archive:  covid_19
  inflating: CovidDataset/Val/Covid/88de9d8c39e946abd495b37cd07d89e5-6531-0.jpg  
  inflating: CovidDataset/Val/Covid/covid-19-pneumonia-8.jpg  
  inflating: CovidDataset/Val/Covid/85E52EB3-56E9-4D67-82DA-DEA247C82886.jpeg  
  inflating: CovidDataset/Val/Covid/4-x-day1.jpg  
  inflating: CovidDataset/Val/Covid/covid-19-pneumonia-53.jpg  
  inflating: CovidDataset/Val/Covid/4e43e48d52c9e2d4c6c1fb9bc1544f_jumbo.jpeg  
  inflating: CovidDataset/Val/Covid/covid-19-pneumonia-35-1.jpg  
  inflating: CovidDataset/Val/Covid/53EC07C9-5CC6-4BE4-9B6F-D7B0D72AAA7E.jpeg  
  inflating: CovidDataset/Val/Covid/6CB4EFC6-68FA-4CD5-940C-BEFA8DAFE9A7.jpeg  
  inflating: CovidDataset/Val/Covid/58cb9263f16e94305c730685358e4e_jumbo.jpeg  
  inflating: CovidDataset/Val/Covid/6b3bdbc31f65230b8cdcc3cef5f8ba8a-40ac-0.jpg  
  inflating: CovidDataset/Val/Covid/auntminnie-d-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg  
  inflating: CovidDataset/Val/Covid/6b44464d-73a7-4cf3-bbb6-ffe7

In [3]:
#CODE
TRAIN_PATH = "CovidDataset/Train"
VAL_PATH = "CovidDataset/Test"

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.layers import *
from keras.models import *
from keras.preprocessing import image

In [5]:
# CNN Based Model
# Its a sequential model, which means it has layers
# Each layer has certain filters. like layer1 has n no of filters, layer2 has m no of filters and so on.

# Here we will be creating 3 or 4 cnn layers followed by classification layers

model = Sequential()
model.add(Conv2D(32, kernel_size=(3,3), activation='relu', input_shape=(224, 224, 3)))

#First of all we added a conv layer with 32 filters, we're keeping no of filters small in the beginning
# lower layers detect features in v small part of image, if u just see a v small part of an img then u will
# make a v few patterns.
# But as you go deep into the network, ur receptive field of cnn layer increases ie the features it extract are 
# are based upon a bigger part of the original image. **CONCEPT OF RECEPTIVE FIELD**


model.add(Conv2D(64, (3, 3), activation='relu'))    # add another conv layer
model.add(MaxPooling2D(pool_size=(2, 2)))           # add maxpooling layer
model.add(Dropout(0.25))
# we havent used vgg bcs it will overfit. watch tut at 57.00

# SECOND conv layer
model.add(Conv2D(64, (3, 3), activation='relu'))    # add another conv layer
model.add(MaxPooling2D(pool_size=(2, 2)))           # add maxpooling layer
model.add(Dropout(0.25))

# THIRD conv layer
model.add(Conv2D(128, (3, 3), activation='relu'))    # add another conv layer
model.add(MaxPooling2D(pool_size=(2, 2)))           # add maxpooling layer
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss=keras.losses.binary_crossentropy, optimizer='adam', metrics=['accuracy'])


model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 54, 54, 64)        0

In [6]:
# Train from scratch
# make the traing_data objects
train_datagen = image.ImageDataGenerator(
    rescale = 1./255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True, 
)

test_dataset = image.ImageDataGenerator(rescale=1./255)


In [7]:

train_generator = train_datagen.flow_from_directory(
    'CovidDataset/Train',
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'binary'
)

Found 224 images belonging to 2 classes.


In [8]:
train_generator.class_indices

{'Covid': 0, 'Normal': 1}

In [9]:
validation_generator = test_dataset.flow_from_directory(
    'CovidDataset/Val',
    target_size = (224, 224),
    batch_size = 32,
    class_mode = 'binary'
)

Found 60 images belonging to 2 classes.


In [10]:
hist = model.fit(
    train_generator,
    steps_per_epoch=6,
    epochs = 8,
    validation_data = validation_generator,
    validation_steps=2
)

Epoch 1/8
6/6 [==============================] - 64s 10s/step - loss: 1.1822 - accuracy: 0.5426 - val_loss: 0.6858 - val_accuracy: 0.5000
Epoch 2/8
6/6 [==============================] - 61s 10s/step - loss: 0.6766 - accuracy: 0.5937 - val_loss: 0.6392 - val_accuracy: 0.8833
Epoch 3/8
6/6 [==============================] - 60s 10s/step - loss: 0.5421 - accuracy: 0.7004 - val_loss: 0.4996 - val_accuracy: 0.9500
Epoch 4/8
6/6 [==============================] - 61s 10s/step - loss: 0.4126 - accuracy: 0.8083 - val_loss: 0.3363 - val_accuracy: 0.9500
Epoch 5/8
6/6 [==============================] - 66s 11s/step - loss: 0.2863 - accuracy: 0.8648 - val_loss: 0.3250 - val_accuracy: 0.9667
Epoch 6/8
6/6 [==============================] - 63s 10s/step - loss: 0.1829 - accuracy: 0.9541 - val_loss: 0.1429 - val_accuracy: 0.9500
Epoch 7/8
6/6 [==============================] - 62s 10s/step - loss: 0.2299 - accuracy: 0.9292 - val_loss: 0.0855 - val_accuracy: 0.9667
Epoch 8/8
6/6 [===================

In [11]:
score = model.evaluate(validation_generator, verbose=0)
print('loss: ', score[0])
print('accuracy: ', score[1])

loss:  0.09220551699399948
accuracy:  0.9833333492279053


In [12]:
# Configuration related preprocessing step before mounting the drive
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 160690 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.26-0ubuntu1~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.26-0ubuntu1~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.26-0ubuntu1~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope

In [13]:
#Mount the google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [14]:
#Save the model online
model.save('/content/drive/MyDrive/Colab Notebooks/covid19detector.h5')

In [15]:
# =================== TESTING PART ======================== #

model1 = load_model('/content/drive/MyDrive/Colab Notebooks/covid19detector.h5')

In [16]:
import os
train_generator.class_indices

{'Covid': 0, 'Normal': 1}

In [31]:
# Import few more necessary libraries.
from keras.models import load_model
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
final_result = ""
# Function to load and prepare the image in right shape
def load_image(filename):
  # Load the image
  img = image.load_img(filename, target_size=(224, 224))
  # Convert the image to array
  img = image.img_to_array(img)
  img = np.expand_dims(img, axis=0)
  # model = load_model('/content/drive/MyDrive/Colab Notebooks/covid19detector.h5') /loaded above/
  p = model.predict(img)
  # Predict the output
  # Map category with the numerical class
  if p[0,0] == 0.0:
    final_result = "Covid Positive"
  elif p[0,0] == 1.0:
    final_result = "Covid Negative"
  print(p[0,0], ":", final_result)

load_image('/content/CovidDataset/Val/Covid/6CB4EFC6-68FA-4CD5-940C-BEFA8DAFE9A7.jpeg')
load_image('/content/CovidDataset/Val/Covid/23E99E2E-447C-46E5-8EB2-D35D12473C39.png')


0.0 : Covid Positive
0.0 : Covid Positive


In [22]:
# Random testing 
# Covid +ve image returns 0 value
img = image.load_img('/content/CovidDataset/Val/Normal/NORMAL2-IM-0395-0001.jpeg', target_size=(224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
p = model.predict(img)
print(p[0,0])

1.0


In [ ]:
# Covid -ve image returns 1 value
img = image.load_img('/content/CovidDataset/Val/Normal/NORMAL2-IM-0395-0001.jpeg', target_size=(224, 224))
img = image.img_to_array(img)
img = np.expand_dims(img, axis=0)
p = model.predict(img)
# y_test.append(p[0,0])
# y_actual.append(1)
print(p[0,0])

if p[0,0] == 0.0:
  final_result = "Covid Positive"
elif p[0,0] == 1.0:
  final_result = "Covid Negative"
print(final_result)